In [ ]:
from flask import Flask, render_template

app=Flask(__name__)


In [ ]:
if  __name__=="__main__":
    app.run(debug=True)

In [1]:
#camera.py
import cv2
from imutils.video import WebcamVideoStream
import pickle

class VideoCamera(obj):
    def __init__(self):
        
        self.stream = WebcamVideoStream(src=1).start()
        #pickle_in = open("model_trained1.p","rb")
        #model = pickle.load(pickle_in)
        
    def __del__(self):
        self.stream.stop()
        
    def get_frame(self):
        image = self.stream.read()
        #idhar call karna he detection 
        ret,jpeg = cv2.imencode('.jpg',image)
        data=[]
        data.append(jpeg.tobytes())
        return data

SyntaxError: unexpected EOF while parsing (<ipython-input-1-de80f83f3e85>, line 4)

In [ ]:
from camera import VideoCamera
import flask

def gen(camera):
    global person_name
    while True:
        data = camera.get_frame()
         frame = data[0]
            yield(b'--frame\r\n
                  b'Content-Type: image\jpeg/r/n/r/n' + frame + b'r\n\r\n')
            
@app.route('/video feed')
def video_feed():
    return Response (gen(VideoCamera()), mimetype='multipart/x-mixed-replace: boundary=frame')
@app.route('/result',methods=["POST","GET"])

In [1]:
import torch

import torch.nn as nn
import numpy as np
import cv2
import torch.nn.functional as F
import time


In [2]:
import pickle
pickle_in = open("model_asl.p","rb")
model = pickle.load(pickle_in)

Using TensorFlow backend.


In [3]:
def hand_area(img):
    hand = img[100:324, 100:324]
    hand = cv2.resize(hand, (224,224))
    return hand

In [4]:
cap = cv2.VideoCapture(0)
if (cap.isOpened() == False):
    print('Error while trying to open camera. Plese check again...')
# get the frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [6]:
# read until end of video
while(cap.isOpened()):
    # capture each frame of the video
    ret, frame = cap.read()
    # get the hand area on the video capture screen
    cv2.rectangle(frame, (100, 100), (324, 324), (20, 34, 255), 2)
    hand = hand_area(frame)
    image = hand
    
#     image = np.transpose(image, (2, 0, 1)).astype(np.float32)
#     image = torch.tensor(image, dtype=torch.float)
#     image = image.unsqueeze(0)
    
    outputs = model(image)
    _, preds = torch.max(outputs.data, 1)
    
    cv2.putText(frame, lb.classes_[preds], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    cv2.imshow('image', frame)
    out.write(frame)
    # press `q` to exit
    if cv2.waitKey(27) & 0xFF == ord('q'):
        break
# release VideoCapture()
cap.release()
# close all frames and video windows
cv2.destroyAllWindows()

ValueError: Layer sequential_1 was called with an input that isn't a symbolic tensor. Received type: <class 'torch.Tensor'>. Full input: [tensor([[[[ 20.,  20.,  20.,  ...,  20.,  20.,  20.],
          [ 20.,  20.,  20.,  ...,  20.,  20.,  20.],
          [ 20.,  20., 215.,  ..., 249., 249.,  20.],
          ...,
          [ 20.,  20., 201.,  ...,  31.,  30.,  20.],
          [ 20.,  20., 199.,  ...,  33.,  32.,  20.],
          [ 20.,  20.,  20.,  ...,  20.,  20.,  20.]],

         [[ 34.,  34.,  34.,  ...,  34.,  34.,  34.],
          [ 34.,  34.,  34.,  ...,  34.,  34.,  34.],
          [ 34.,  34., 214.,  ..., 246., 246.,  34.],
          ...,
          [ 34.,  34., 206.,  ...,  33.,  34.,  34.],
          [ 34.,  34., 207.,  ...,  44.,  41.,  34.],
          [ 34.,  34.,  34.,  ...,  34.,  34.,  34.]],

         [[255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 190.,  ..., 224., 222., 255.],
          ...,
   

In [1]:
import numpy as np
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import pickle

Using TensorFlow backend.


In [2]:
pickle_in = open("model_trained.p","rb")
model = pickle.load(pickle_in)
background = None
accumulated_weight = 0.5

ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

In [3]:
#Function to calculate the background accumulated weighted average
def cal_accum_avg(frame, accumulated_weight):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
#Segmenting the hand, i.e, getting the max contours and the thresholded image of the hand detected.
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)

    
    _ , thresholded = cv2.threshold(diff, threshold, 255,cv2.THRESH_BINARY)
    
     #Fetching contours in the frame (These contours can be of hand or any other object in foreground) …

    image, contours, hierarchy =cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    # If length of contours list = 0, means we didn't get anycontours...
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand 
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        # Returning the hand segment(max contour) and thethresholded image of hand...
        return (thresholded, hand_segment_max_cont)

In [7]:
cam = cv2.VideoCapture(0)
num_frames =0
while True:
    ret, frame = cam.read()

    # flipping the frame to prevent inverted image of capturedframe...
    
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    # ROI from the frame
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)


    if num_frames < 70:
        
        cal_accum_avg(gray_frame, accumulated_weight)
        
        cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT",(80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else: 
        # segmenting the hand region
        hand = segment_hand(gray_frame)
        
        # Checking if we are able to detect the hand...
        if hand is not None:
            
            thresholded, hand_segment = hand

            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right,ROI_top)], -1, (255, 0, 0),1)
            
            cv2.imshow("Thesholded Hand Image", thresholded)
            
            thresholded = cv2.resize(thresholded, (64, 64))
            thresholded = cv2.cvtColor(thresholded,cv2.COLOR_GRAY2RGB)
            thresholded = np.reshape(thresholded,(1,thresholded.shape[0],thresholded.shape[1],3))
            
            pred = model.predict(thresholded)
            cv2.putText(frame_copy, word_dict[np.argmax(pred)],(170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
    # Draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,ROI_bottom), (255,128,0), 3)

    # incrementing the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.putText(frame_copy, "DataFlair hand sign recognition_ _ _",(10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Detection", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()

ValueError: Error when checking input: expected conv2d_1_input to have shape (32, 32, 1) but got array with shape (64, 64, 3)

In [1]:
import cv2
import numpy as np
import pickle
pickle_in = open("model_asl.h5","rb")
model = pickle.load(pickle_in)

Using TensorFlow backend.


In [2]:
labels = list("ABC")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        continue
    
    cv2.rectangle(frame,(300,0),(500,200),(255,0,0),thickness=1)
    
    img = frame[0:200,300:500]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (100,100))
    cv2.imshow('ASL_Hand',img)
    img = np.reshape(img, (1,100,100,1))
    try:
        predictions = model.predict(img)
        print(predictions)
        cv2.putText(frame, labels[np.argmax(predictions)], (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    except:
        pass

    resized_img = cv2.resize(frame, (1200, 700))  
    cv2.imshow('ASL',resized_img)

    if cv2.waitKey(10) == ord('s'): 
            break  

cap.release()  
cv2.destroyAllWindows()